In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
%load_ext watermark
%watermark -v -p numpy,pandas,scipy,scikit-learn,torch,rdkit,gpytorch,matplotlib,botorch,wandb


Python implementation: CPython
Python version       : 3.8.16
IPython version      : 8.8.0

numpy       : 1.23.5
pandas      : 1.5.3
scipy       : 1.10.1
scikit-learn: 1.2.2
torch       : 2.0.1
rdkit       : 2023.3.1
gpytorch    : 1.10
matplotlib  : 3.3.2
botorch     : 0.8.2.dev9+g7f3aa92f
wandb       : 0.15.3



In [3]:
from chaos.data.module import BaseDataModule, Featurizer
from chaos.bo.module import BoModule
from chaos.initialization.initializers import BOInitializer
from pytorch_lightning import seed_everything
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from chaos.utils import instantiate_class

To use the Graphein submodule graphein.protein.features.sequence.embeddings, you need to install: biovec 
biovec cannot be installed via conda
To use the Graphein submodule graphein.protein.visualisation, you need to install: pytorch3d 
To do so, use the following command: conda install -c pytorch3d pytorch3d


[09/08/23 13:53:19] WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=728192;file:///home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=841483;file:///home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

# CHAOS Tutorial

Welcome to CHAOS! (tutorial) We will walk through the entire workflow of the CHAOS framework, starting with loading your dataset from a CSV file to running a Bayesian optimization loop to find the optimal experimental settings for your chemical reactions.

## Introduction

CHAOS, which stands for CHemical Additives Optimization Screening is an open-source framework that leverages Bayesian optimization and machine learning to facilitate the optimization of chemical reactions. The objective of this tutorial is to provide a hands-on guide to using CHAOS for chemical reaction optimization.


## Data loading and featurization

CHAOS provides an easy way to featurize molecules or reactions using a variety or representations. Depending on the type of the task (molecular vs reaction optimization) you can set up the Featurizer. In the following cells we demonstrate how to set up a featurizer using DRFP representations for reaction optimization, or fragprints for molecular optimization.

In [4]:
## Fragprints
featurizer = Featurizer(
    nBits=512, bond_radius=3, representation="fragprints", task="molecular_optimization"
)
## DRFP
featurizer = Featurizer(
    nBits=512, bond_radius=7, representation="drfp", task="reaction_optimization"
)


If you want to employ a specific initialization, you can use the BOInitializer object. Depending on the method, it will provide selection using either clustering, maxmin strategy or random sampling. We show how to set each of these options. 

In [5]:
## Kmeans
initializer = BOInitializer(method="kmeans", n_clusters=10, use_pca=10)
## Kmedoids
initializer = BOInitializer(method="kmedoids", n_clusters=10, metric="jaccard")
## MaxMin
initializer = BOInitializer(method="maxmin", n_clusters=10, metric="jaccard")
## Random
initializer = BOInitializer(method="true_random", n_clusters=10, metric="jaccard")



Finally we can employ the data module, responsible for storing the featurized data, preprocessing it and splitting into the train and heldout sets. It takes as input the path to a .csv file containing smiles of the reaction components to be screened. The *input_column* is the column of the .csv file that contains the specific component for optimization. For molecular optimization, as in the paper, this would be a column where the set of additive smiles are stored. For reaction representation, the column needs to contain reaction smiles. In the case of OHE you can forward a list of columns, and the featurization would create a unifying OHE vector based on those columns.

The target column is the column containing the objective values. Additionally it takes as input initializer and featurizer objects. 

In [6]:
dm = BaseDataModule(
    data_path="../data/additives/additive_rxn_screening_plate_1.csv",
    input_column="rxn",
    target_column="objective",
    initializer=initializer,
    featurizer=featurizer,
)

[653, 394, 545, 422, 171, 654, 446, 522, 353, 374] selected reactions
Selected reactions: [653, 394, 545, 422, 171, 654, 446, 522, 353, 374]


In [7]:
dm.train_x.shape, dm.train_y.shape, dm.heldout_x.shape, dm.heldout_y.shape, dm.x.shape, dm.y.shape

(torch.Size([10, 512]),
 torch.Size([10, 1]),
 torch.Size([710, 512]),
 torch.Size([710, 1]),
 torch.Size([720, 512]),
 torch.Size([720, 1]))

## Setting up the surrogate model

The surrogate model can be initialized based on a formatted config defining the necessary arguments like the specific kernel, noise constraints and whether to standardize the output. If you want to change the kernel, just put edit the path to the kernel class, for example *gpytorch.kernels.LinearKernel* or custom kernels, for example from GauChe, you can use them here. 


In [8]:
model_config = {
    "class_path": "chaos.surrogate_models.gp.SimpleGP",
    "init_args": {
        "likelihood": {
            "class_path": "gpytorch.likelihoods.GaussianLikelihood",
        },
        "covar_module": {
            "class_path": "gpytorch.kernels.ScaleKernel",
            "init_args": {
                "base_kernel": {
                    "class_path": "gpytorch.kernels.MaternKernel",
                    "init_args": {"eps": 1.0e-06, "nu": 0.5},
                },
                "eps": 1.0e-06,
            },
        },
        "standardize": True,
        "normalize": False,
        "initial_noise_val": 0.0001,
        "noise_constraint": 1.0e-05,
        "initial_outputscale_val": 2.0,
        "initial_lengthscale_val": 0.5,
    },
}

In [9]:
gp = instantiate_class(model_config, train_x=dm.train_x, train_y=dm.train_y)

## Setting up the BO loop
The main module for Bayesian optimization is BoModule with primary inputs like the data and the model_config. We can use it inside the pytorch lightning Trainer object to run the BO iterations. All the metrics are saved to wandb.

In [10]:
bo_module = BoModule(
        data=dm,
        model_config=model_config,
        enable_plotting=True,
        enable_logging_images=True,
        beta=0.1,
    )

In [11]:
logger = (
    WandbLogger(project="chaos-tutorial") if bo_module.enable_plotting else None
)
trainer = Trainer(
    max_epochs=100,
    logger=logger,
    log_every_n_steps=1,
    num_sanity_val_steps=0,
    min_epochs=1,
    max_steps=-1,
    accelerator="cpu",
    devices=1,
)
trainer.fit(bo_module)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bojana-rankovic (liac). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
/home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/pytorch_lightning/core/optimizer.py:171: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name | Type | Params
--

Training: 0it [00:00, ?it/s]

/home/rankovic/miniconda3/envs/additive_bo/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('suggestion_idx', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=100` reached.


MAE_all,█▄▂▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▃▂
MAE_bottom_5,▁▄▆▇▆▆▇▆▇█▇▇▇▇▆▅▅▅▅▆▆▆▆▆▆▅▆▆▆▆▆▆▆▆▇▇▆▆▇▆
MAE_top_5,█▅▃▂▃▃▂▃▂▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▃▂▂▂
NLPD_all,█▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
NLPD_bottom_5,█▄▁▁▁▁▂▂▃▄▃▄▄▄▄▃▄▄▄▄▄▅▄▄▄▄▅▅▆▅▆▆▆▇▇▇▇▆▆▅
NLPD_top_5,█▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
R2_all,▁▆████▇▇▇▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆
R2_bottom_5,█▇▅▅▅▄▄▄▄▃▄▄▅▄▄▅▅▅▅▅▅▄▄▄▄▄▄▄▂▃▂▂▂▂▂▂▁▃▂▄
R2_top_5,▃▅▇▇▇▇█▇██▇▇▇▇▇▇▆▇▆▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▄▁▁
average_similarity,▁▁▄▆▆▇▇▇██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇
covar_module.base_kernel.lengthscale,▁▂▄▆▄▅▆▅▆█▅▆▅▅▄▄▄▃▃▃▃▄▄▄▄▄▃▃▄▄▄▄▄▄▄▄▄▄▄▃
